In [ ]:
import os
key = os.environ.get('AZURE_SEARCH_KEY')
print(key)

In [ ]:
print(key)

In [ ]:
from utils import search_images_bing

search_images_bing


In [ ]:
print(key)


In [ ]:
results = search_images_bing(key, 'christmas mug')
print(key)
ims = results.attrgot('contentUrl')
len(ims)

In [ ]:
print(results[0].keys())

In [ ]:
from fastai.data.external import *
from fastai.data.transforms import *


In [ ]:
dest = 'images/mug_christmas.jpg'
download_url(ims[0], dest)
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
dir_n_search = {
    'christmas': 'christmas mug',
    'pottery': 'pottery mug',
    'kids': 'kids mug',
    'straw': 'mug with straw',
    'teacup': 'tea cup'
}

In [ ]:
path = Path('00-mugs')
print(key)
if not path.exists():
    path.mkdir()
if True:
    for k, v in dir_n_search.items():
        print(f'{k} => {v}')
        dest = (path/k)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{v}')
        download_images(dest, urls=results.attrgot('contentUrl'))
    

In [ ]:
from fastai.data.transforms import *
fns = get_image_files(path)

In [ ]:
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)


In [ ]:
len(fns)

In [ ]:
?verify_images

In [ ]:
mugs = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

In [ ]:
mugs

In [ ]:
dls = mugs.dataloaders(path)

In [ ]:
dls

In [ ]:
?dls


In [ ]:
??dls


In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
mugs2 = mugs.new(item_tfms=RandomResizedCrop(224, min_scale=0.5),
                batch_tfms=aug_transforms())
dls = mugs2.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)